In [2]:
from torch.utils.data import Dataset
import torch
import torch.nn as nn
from d2l import torch as d2l
from more.datasets import ModulationDataSets
from more.network import MoreNet,init_weights
from more.train import train_center
from utils.centerloss import CenterLoss
import datetime
import time

In [3]:
train_path = r'E:\train.h5'
test_path = r'E:\test.h5'
train_dataset = ModulationDataSets(train_path)
test_dataset = ModulationDataSets(test_path)
label_name = train_dataset.get_labels_name();

In [4]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=0, drop_last=False)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset, batch_size=batch_size, shuffle=True,
    num_workers=0, drop_last=False)

In [5]:
net = MoreNet()
X = torch.rand(size=(1, 1, 196, 64), dtype=torch.float32)
net(X,True)

E:\workspace\DeepMore\more\network.py:52: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return x.detach(), F.softmax(y)


(tensor([[-0.0021, -0.0019]]),
 tensor([[0.1256, 0.0826, 0.0701, 0.0906, 0.1345, 0.2531, 0.0773, 0.1662]],
        grad_fn=<SoftmaxBackward0>))

In [6]:
net.load_state_dict(torch.load('basic-8.parm'))

<All keys matched successfully>

In [ ]:
torch.save(net.state_dict(),"basic-8.parm")

In [7]:
init_weights(net)

In [8]:
loss= nn.CrossEntropyLoss()
center_loss=CenterLoss(num_classes=train_dataset.get_numclasses(),feat_dim=2,use_gpu=True)
lr=0.01
lr_center=0.5
opt_model=torch.optim.SGD(net.parameters(),lr,weight_decay=5e-04, momentum=0.9)
opt_center=torch.optim.SGD(center_loss.parameters(),lr_center)
net=net.to("cuda:0")

In [9]:
start_time = time.time()

for epoch in range(50,100):
    print("==> Epoch {}/{}".format(epoch+1,50))

    train_center(net, loss, center_loss, opt_model, opt_center, train_loader, True, train_dataset.get_numclasses(), epoch)
    #
    # if  (epoch+1) % 10 == 0 or (epoch+1) == 50:
    #     print("==> Test")
    #     acc, err = test(model, testloader, use_gpu, dataset.num_classes, epoch)
    #     print("Accuracy (%): {}\t Error rate (%): {}".format(acc, err))

elapsed = round(time.time() - start_time)
elapsed = str(datetime.timedelta(seconds=elapsed))
print("Finished. Total elapsed time (h:m:s): {}".format(elapsed))

==> Epoch 1/50


E:\workspace\DeepMore\utils\centerloss.py:35: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1055.)
  distmat.addmm_(1, -2, x, self.centers.t())


Batch 50/313	 Loss 2.083141 (2.350923) XentLoss 2.079204 (2.081535) CenterLoss 0.003937 (0.269388)
Batch 100/313	 Loss 2.087080 (2.218107) XentLoss 2.080507 (2.081040) CenterLoss 0.006574 (0.137067)
Batch 150/313	 Loss 2.330319 (2.183838) XentLoss 2.087688 (2.079568) CenterLoss 0.242631 (0.104270)
Batch 200/313	 Loss 5.436439 (2.559732) XentLoss 2.028309 (2.070622) CenterLoss 3.408130 (0.489110)
Batch 250/313	 Loss 15.665959 (4.438256) XentLoss 1.952273 (2.055582) CenterLoss 13.713686 (2.382673)
Batch 300/313	 Loss 51.110764 (8.742311) XentLoss 1.912423 (2.040556) CenterLoss 49.198341 (6.701755)
==> Epoch 2/50
Batch 50/313	 Loss 195.854492 (75.217459) XentLoss 1.916459 (1.917721) CenterLoss 193.938034 (73.299738)
Batch 100/313	 Loss 171.563431 (134.402029) XentLoss 1.861558 (1.917081) CenterLoss 169.701874 (132.484947)
Batch 150/313	 Loss 167.416840 (202.418037) XentLoss 1.948418 (1.918599) CenterLoss 165.468414 (200.499439)
Batch 200/313	 Loss 525.093262 (223.447275) XentLoss 1.912131

In [10]:
torch.save(net.state_dict(),"basic-8.parm")